# Práctica 1: Búsquedas en espacios de estados

<center><h3>
   Marta Folgar Martínez
</h3></center>


## El problema de Viajante de Comercio (VC)

El objetivo de este bloque es modelar e implementar un agente inteligente que sea capaz de resolver
el problema del VC mediante estrategias de búsqueda A*, haciendo uso de AIMA.


### Definición del problema

El problema del viajante de comercio (VC) es el problema de la persona que quiere vender un producto, y que para ello quiere encontrar el viaje más corto posible a través de las ciudades de los clientes, haciendo una única visita a cada una, empezando y acabando el recorrido en su propia ciudad (recorrido circular desde la ciudad inicial).

Típicamente, el problema parte de una representación mediante un grafo ponderado G=(N, A), donde N es el conjunto de n=|N| nodos (ciudades), y siendo A el conjunto de arcos conectando los nodos. Cada arco (i, j) ∈ A tiene asignado un peso d_ij que representa la distancia entre las ciudades i y j.

Se proporciona la clase `Localizaciones`, que permite cargar las localizaciones GPS que representan los vértices del grafo G de N ciudades, y permite calcular de manera transparente la distancia entre cualquier par de ciudades usando la fórmula del semiverseno (https://es.wikipedia.org/wiki/F%C3%B3rmula_del_semiverseno), que sirve para calcular las distancias teniendo en cuenta la curvatura de la Tierra.


In [17]:
# contiene algoritmos de búsqueda implementado en AIMA
from search_mod import *

# algunas funciones auxiliares
from helpers_mod import *

In [18]:
 psource(Problem)

In [19]:
psource (Localizaciones)

Por defecto se carga el fichero `./data/grafo8cidades.txt`, que contiene las coordenadas GPS de 8 ciudades gallegas, siendo Santiago de Compostela la primera de ellas. La primera línea de estos ficheros indica el número de ciudades n, mientras que cada una de las líneas sucesivas especifican las coordenadas de cada ciudad, especificadas como coordenadas GPS (latitud y longitud en grados).

Puedes cargar otro fichero haciendo uso del parámetro `filename` como se muestra a continuación.

### Implementación del problema VC


El VC se reduce al problema de crear el circuito Hamiltoniano de longitud mínima sobre el grafo G. La solución a una instancia del problema del VC puede representarse como una permutación de los índices de las ciudades, donde lo importante es el orden de visita, que determinará el coste del viaje en términos de la distancia recorrida total. 

De este modo, el problema pertenece a la categoría de problemas NP, pues puede haber n permutaciones que se corresponden al espacio de búsqueda posible. Esto hace que resolver instancias de problemas con muchas ciudades (n grande) haga el problema impracticable y éste pueda beneficiarse de búsquedas informadas, que guíen inteligentemente el proceso para reducir el espacio de búsqueda y por tanto el esfuerzo computacional.


A continuación se ha implementado el **problema del Viajande de Comercio**. Para ello se ha implementado la clase `ProblemaViajanteComercio`, la cual extiende la clase `Problem` y sobreescribe los métodos `actions, result, is_goal, action_cost`.

En esta primera propuesta se considera una implementación nula de la heurística, h=0, de forma que debería funcionar como una búsqueda de coste uniforme. 

### Implementación H=0

In [20]:
class ProblemaViajanteComercio(Problem):
    """ Problema Viajante de Comercio"""
    
    def __init__(self, localizaciones):
        self.localizaciones = localizaciones
        self.initial = [0] #se empieza el recorrido en la ciudad 0
                 
        lista=list(self.initial) #la tupla del estado inicial se completa con -1 para el resto de ciudades (0,-1,-1,...)                
        for i in range (self.localizaciones.nciudades):
            lista.append(-1) 
        self.initial=tuple(lista)
        
            
    def actions(self, state):
        """devuelve la lista de ciudades no visitadas como acciones potenciales."""     
        #cuando ya se han visitado todas las ciudades, hay que volver a la inicial
        if (state.index(-1)==self.localizaciones.nciudades): 
            return tuple([0]) #la única acción posible es volver a la ciudad inicial
        
        lista=[]
        ciudad=state[state.index(-1)-1] #se obtiene la última ciudad visitada. Ej: (0,3,5,-1,-1,-1)->5     
        for i in range(self.localizaciones.nciudades):
            if i!=ciudad and i not in state: #ciudades que aún no se han visitado
                lista.append(i)
                
        actions=tuple(lista)   
        return actions #se devuelven todas las posibles ciudades a las que se puede ir
    
    
    def result(self, state, action):
        """produce el nuevo estado al anadir la accion seleccionada"""   
        lista=list(state) 
        lista[state.index(-1)]=action #la accion indica la ciudad a la que se debe ir
        result=tuple(lista)
        
        return result
    
    
    def is_goal(self, state):            
        cont=0
        for i in range(self.localizaciones.nciudades):  #se comprueba que se hayan visitado todas las ciudades
            if i in state:
                cont+=1
              
        #el problema está resuelto si se visitaron todas las ciudades y se ha vuelto a la ciudad inicial
        if cont==(self.localizaciones.nciudades) and (state[0]==state[self.localizaciones.nciudades]):  
            return True
        return False
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""  
        
        if -1 in s1: 
            #se devuelve la distancia entre las dos últimas ciudades
            #s tendrá la forma: (n1, n2, n3, -1, -1...)
            #s1 tendrá la forma: (n1, n2, n3, n4, -1...)
            #por lo tanto se devolverá la distancia entre n3 y n4
            return self.localizaciones.distancia(s1[s1.index(-1)-2],s1[s1.index(-1)-1]) 
        else:
            #si ya se han visitado todas las ciudades, se devuelve la distancia entre la última y el regreso a la ciudad inicial
            return self.localizaciones.distancia(s1[self.localizaciones.nciudades-1],s1[self.localizaciones.nciudades]) 
        
    def h(self, node): 
        return 0 #en esta implementación la heurística es nula

    
    
g1=Localizaciones(filename='./data/grafo8cidades.txt')
VC1=ProblemaViajanteComercio(g1)
sol = astar_search(VC1)
print ("Nodo solución: ", sol)
print (f'Coste del camino: {sol.path_cost:f}')
print (f'Lista de acciones: {path_actions(sol)}')
print (f'Numero de acciones en el camino: {len(path_actions(sol))}')
print (f'Lista de estados:')
print (path_states(sol))    

Nodo solución:  <(0, 1, 2, 3, 4, 5, 6, 7, 0)>
Coste del camino: 381.669962
Lista de acciones: [1, 2, 3, 4, 5, 6, 7, 0]
Numero de acciones en el camino: 8
Lista de estados:
[(0, -1, -1, -1, -1, -1, -1, -1, -1), (0, 1, -1, -1, -1, -1, -1, -1, -1), (0, 1, 2, -1, -1, -1, -1, -1, -1), (0, 1, 2, 3, -1, -1, -1, -1, -1), (0, 1, 2, 3, 4, -1, -1, -1, -1), (0, 1, 2, 3, 4, 5, -1, -1, -1), (0, 1, 2, 3, 4, 5, 6, -1, -1), (0, 1, 2, 3, 4, 5, 6, 7, -1), (0, 1, 2, 3, 4, 5, 6, 7, 0)]


En esta implementación, el estado se representa mediante una tupla de tamaño nciudades+1 (debido a que una vez se visiten todas las ciudades, hay que regresar a la ciudad inicial). En la tupla, el valor de cada posición se corresponderá con el número de cada ciudad visitada y se marcarán con un -1 aquellas ciudades que aún no se han visitado. De esta forma, esta tupla de tamaño 7 (0,4,5,3,-1,-1,-1) indica que hay 6 ciudades que se deben visitar y que de momento sólo se han visitado las ciudades 0, 4, 5 y 3.


En la implementación de mi función "actions", en primer lugar se comprueba si ya se han visitado todas las ciudades y en ese caso la única acción posible será regresar a la ciudad inicial. En caso contrario, se devuelven las ciudades que aún no se han visitado.

La función "result" actualiza el estado cambiando el -1 que sigue a la última ciudad visitada con la ciudad a la que se irá a continuación. De este modo, si estamos en el estado (0,3,4,-1,-1,-1) y la acción es 5, la función result devolverá el estado (0,3,4,5,-1,-1).

La función "is_goal" comprueba si se han recorrido ya todas las ciudades y si se ha regresado a la ciudad inicial. En ese caso, se devolverá True y finalizará la ejecución del algoritmo, y en caso contrario False.

Por último, la función "action_cost" devuelve la distancia entre el último trayecto entre ciudades visitadas. El estado s que se pasa como argumento tendrá la forma (0,3,4,-1,-1) y s1 (0,3,4,6,-1), es decir, s1 incluye la ciudad a la que se ha ido desde la última visitada en s, por lo tanto la función devolverá el trayecto entre las dos últimas ciudades de s1.


### Implementación de heurísticas para VC

La elección de heurísticas que estimen el coste de alcanzar un estado objetivo a partir de un estado dado es crucial para que la estrategia de búsqueda A* sea guiada con mayor eficacia. Recordemos que cuanta más precisa sea la estimación de la función h (i.e., más cercana al coste real), más informada será la búsqueda. Es conveniente que la heurística subestime el coste real para asegurar que la estrategia sigue siendo admisible, y por tanto no descarte estados que puedan conducir a la solución óptima.

A continuación se incorporan **tres heurísticas diferentes** a la implementación A* del VC.

    - h1 (n) = vecino más próximo: esta heurística considera como coste estimado la distancia al vecino más próximo que no haya sido visitado todavía, persiguiendo de esta forma que se produzca el mínimo incremento en longitud al añadir la ciudad al camino.
    - h2 (n) = vecino más lejano: esta heurística considera como coste estimado la distancia al vecino más lejano que no haya sido visitado todavía, teniendo en cuenta de esta forma posible coste en que se podría incurrir luego por esos nodos más lejanos pendientes.
    - h3 (n) = distancias medias: esta heurística considera realizar una estimación calculada como la distancia media a los nodos pendientes de visitar.

### Implementación H1(n) - Vecino más próximo

La primera de las heurísticas, la del vecino más próximo, se ha implementado de la siguiente manera:

Dado un estado, su función heurística h(n) será la distancia a la ciudad más próxima de las que todavía no se han visitado. 
Para ello, en primer lugar se comprobará si ya se ha terminado el recorrido o no, es decir, si ya se han visitado todas las ciudades y regresado a la ciudad inicial. En caso afirmativo, se devolverá un valor 0. En caso contrario, pueden darse dos casos: 
- Que se hayan visitado todas las ciudades pero que falte regresar a la ciudad inicial: en ese caso hay una única opción de movimiento y h(n) será la distancia desde la última ciudad visitada a la ciudad de partida.
- Que aún falten ciudades por visitar: en ese caso, se tomará la última ciudad visitada y h(n) será el mínimo de las distancias desde esa ciudad a todas las que aún faltan por visitar. Es decir, si actualmente nos encontramos en la ciudad 1: (0,1,-1,-1,-1,-1,-1,-1,-1), su función heurística será el mínimo de las distancias de [(1,2), (1,3), (1,4), (1,5), (1,6), (1,7), (1,8)].

In [21]:
class ProblemaViajanteComercioH1(Problem):
    """ Problema Viajante de Comercio"""
    
    def __init__(self, localizaciones):
        self.localizaciones = localizaciones
        self.initial = [0] #se empieza el recorrido en la ciudad 0
                 
        lista=list(self.initial) #la tupla del estado inicial se completa con -1 para el resto de ciudades (0,-1,-1,...)                
        for i in range (self.localizaciones.nciudades):
            lista.append(-1) 
        self.initial=tuple(lista)
        
            
    def actions(self, state):
        """devuelve la lista de ciudades no visitadas como acciones potenciales."""     
        #cuando ya se han visitado todas las ciudades, hay que volver a la inicial
        if (state.index(-1)==self.localizaciones.nciudades): 
            return tuple([0]) #la única acción posible es volver a la ciudad inicial
        
        lista=[]
        ciudad=state[state.index(-1)-1] #se obtiene la última ciudad visitada. Ej: (0,3,5,-1,-1,-1)->5     
        for i in range(self.localizaciones.nciudades):
            if i!=ciudad and i not in state: #ciudades que aún no se han visitado
                lista.append(i)
                
        actions=tuple(lista)   
        return actions #se devuelven todas las posibles ciudades a las que se puede ir
               
    
    
    def result(self, state, action):
        """produce el nuevo estado al anadir la accion seleccionada"""   
        lista=list(state) 
        lista[state.index(-1)]=action #la accion indica la ciudad a la que se debe ir
        result=tuple(lista)
        
        return result
    
    
    def is_goal(self, state):        
        cont=0
        for i in range(self.localizaciones.nciudades):  #se comprueba que se hayan visitado todas las ciudades
            if i in state:
                cont+=1
              
        #el problema está resuelto si se visitaron todas las ciudades y se ha vuelto a la ciudad inicial
        if cont==(self.localizaciones.nciudades) and (state[0]==state[self.localizaciones.nciudades]):  
            return True
        return False
    
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""     
        if -1 in s1: 
            #se devuelve la distancia entre las dos últimas ciudades
            #s tendrá la forma: (n1, n2, n3, -1, -1...) y la acción indicará que se vaya a n4, por lo tanto:
            #s1 tendrá la forma: (n1, n2, n3, n4, -1...)
            #y lo que se hace es devolver la distancia entre n3 y n4
            return self.localizaciones.distancia(s1[s1.index(-1)-2],s1[s1.index(-1)-1]) 
        else:
            #si ya se han visitado todas las ciudades, se devuelve la distancia entre la última y el regreso a la ciudad inicial
            return self.localizaciones.distancia(s1[self.localizaciones.nciudades-1],s1[self.localizaciones.nciudades]) 
        
    
    
    def h(self, node): 
        node=node.state

        if -1 in node: #se comprueba si el recorrido ya ha finalizado
            if node.index(-1)==self.localizaciones.nciudades: #se comprueba si se han visitado ya todas las ciudades y solo falta volver a la inicial
                return self.localizaciones.distancia(node[node.index(-1)-1], 0)

            else: 
                ciudad=node[node.index(-1)-1] #se obtiene la ciudad que se está probando
                distancias={}
                for i in range(1, self.localizaciones.nciudades):
                    if i not in node:
                        distancias[i]=self.localizaciones.distancia(ciudad, i) #se obtienen las distancias entre la ciudad que se está probando y las que todavía quedan por visitar

                minimo= min(distancias.values())  #se obtiene el mínimo de todas esas distancias
                return minimo

        else:
            return 0     
          
    
               
g1=Localizaciones(filename='./data/grafo8cidades.txt')
VC1=ProblemaViajanteComercioH1(g1)
sol = astar_search(VC1)
print (f'\n\nCoste del camino: {sol.path_cost:f}')
print (f'Estado solución: {sol.state:}')
print (f'Lista de acciones: {path_actions(sol)}')
print (f'Numero de acciones en el camino: {len(path_actions(sol))}')
print (f'Lista de estados:')
print (path_states(sol))   
        



Coste del camino: 381.669962
Estado solución: (0, 7, 6, 5, 4, 3, 2, 1, 0)
Lista de acciones: [7, 6, 5, 4, 3, 2, 1, 0]
Numero de acciones en el camino: 8
Lista de estados:
[(0, -1, -1, -1, -1, -1, -1, -1, -1), (0, 7, -1, -1, -1, -1, -1, -1, -1), (0, 7, 6, -1, -1, -1, -1, -1, -1), (0, 7, 6, 5, -1, -1, -1, -1, -1), (0, 7, 6, 5, 4, -1, -1, -1, -1), (0, 7, 6, 5, 4, 3, -1, -1, -1), (0, 7, 6, 5, 4, 3, 2, -1, -1), (0, 7, 6, 5, 4, 3, 2, 1, -1), (0, 7, 6, 5, 4, 3, 2, 1, 0)]


### Implementación H2(n) - Vecino más lejano

La segunda de las heurísticas, la del vecino más lejano, se ha implementado de la siguiente manera:

Dado un estado, su función heurística h(n) será la distancia a la ciudad más lejana de las que todavía no se han visitado. 
Para ello, en primer lugar se comprobará si ya se ha terminado el recorrido o no, es decir, si ya se han visitado todas las ciudades y regresado a la ciudad inicial. En caso afirmativo, se devolverá un valor 0. En caso contrario, pueden darse dos casos: 
- Que se hayan visitado todas las ciudades pero que falte regresar a la ciudad inicial: en ese caso hay una única opción de movimiento y h(n) será la distancia desde la última ciudad visitada a la ciudad de partida.
- Que aún falten ciudades por visitar: en ese caso, se tomará la última ciudad visitada y h(n) será el máximo de las distancias desde esa ciudad a todas las que aún faltan por visitar. Es decir, si actualmente nos encontramos en la ciudad 1: (0,1,-1,-1,-1,-1,-1,-1,-1), su función heurística será el máximo de las distancias de [(1,2), (1,3), (1,4), (1,5), (1,6), (1,7), (1,8)].

In [22]:
class ProblemaViajanteComercioH2(Problem):
    """ Problema Viajante de Comercio"""
    
    def __init__(self, localizaciones):
        self.localizaciones = localizaciones
        self.initial = [0] #se empieza el recorrido en la ciudad 0
                 
        lista=list(self.initial) #la tupla del estado inicial se completa con -1 para el resto de ciudades (0,-1,-1,...)                
        for i in range (self.localizaciones.nciudades):
            lista.append(-1) 
        self.initial=tuple(lista)
        
            
    def actions(self, state):
        """devuelve la lista de ciudades no visitadas como acciones potenciales."""     
        #cuando ya se han visitado todas las ciudades, hay que volver a la inicial
        if (state.index(-1)==self.localizaciones.nciudades): 
            return tuple([0]) #la única acción posible es volver a la ciudad inicial
        
        lista=[]
        ciudad=state[state.index(-1)-1] #se obtiene la última ciudad visitada. Ej: (0,3,5,-1,-1,-1)->5     
        for i in range(self.localizaciones.nciudades):
            if i!=ciudad and i not in state: #ciudades que aún no se han visitado
                lista.append(i)
                
        actions=tuple(lista)   
        return actions #se devuelven todas las posibles ciudades a las que se puede ir
               
    
    
    def result(self, state, action):
        """produce el nuevo estado al anadir la accion seleccionada"""   
        lista=list(state) 
        lista[state.index(-1)]=action #la accion indica la ciudad a la que se debe ir
        result=tuple(lista)
        
        return result
    
    
    def is_goal(self, state):        
        cont=0
        for i in range(self.localizaciones.nciudades):  #se comprueba que se hayan visitado todas las ciudades
            if i in state:
                cont+=1
              
        #el problema está resuelto si se visitaron todas las ciudades y se ha vuelto a la ciudad inicial
        if cont==(self.localizaciones.nciudades) and (state[0]==state[self.localizaciones.nciudades]):  
            return True
        return False
    
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""     
        if -1 in s1: 
            #se devuelve la distancia entre las dos últimas ciudades
            #s tendrá la forma: (n1, n2, n3, -1, -1...) y la acción indicará que se vaya a n4, por lo tanto:
            #s1 tendrá la forma: (n1, n2, n3, n4, -1...)
            #y lo que se hace es devolver la distancia entre n3 y n4
            return self.localizaciones.distancia(s1[s1.index(-1)-2],s1[s1.index(-1)-1]) 
        else:
            #si ya se han visitado todas las ciudades, se devuelve la distancia entre la última y el regreso a la ciudad inicial
            return self.localizaciones.distancia(s1[self.localizaciones.nciudades-1],s1[self.localizaciones.nciudades]) 
        
        
    def h(self, node): 
        node=node.state

        if -1 in node: #se comprueba si el recorrido ya ha finalizado
            if node.index(-1)==self.localizaciones.nciudades:  #se comprueba si se han visitado ya todas las ciudades y solo falta volver a la inicial
                return self.localizaciones.distancia(node[node.index(-1)-1], 0) 

            else: 
                ciudad=node[node.index(-1)-1] #se obtiene la ciduad que se está probando
                distancias={}
                for i in range(1, self.localizaciones.nciudades):
                    if i not in node:
                        distancias[i]=self.localizaciones.distancia(ciudad, i) #se obtienen las distancias entre la ciudad que se está probando y las que todavía quedan por visitar

                maximo= max(distancias.values())  #se obtiene el máximo de todas esas distancias
                return maximo
                

        else:
            return 0     
            
         
    
               
g1=Localizaciones(filename='./data/grafo8cidades.txt')
VC1=ProblemaViajanteComercioH2(g1)
sol = astar_search(VC1)
print (f'\n\nCoste del camino: {sol.path_cost:f}')
print (f'Estado solución: {sol.state:}')
print (f'Lista de acciones: {path_actions(sol)}')
print (f'Numero de acciones en el camino: {len(path_actions(sol))}')
print (f'Lista de estados:')
print (path_states(sol))
     
        



Coste del camino: 381.669962
Estado solución: (0, 7, 6, 5, 4, 3, 2, 1, 0)
Lista de acciones: [7, 6, 5, 4, 3, 2, 1, 0]
Numero de acciones en el camino: 8
Lista de estados:
[(0, -1, -1, -1, -1, -1, -1, -1, -1), (0, 7, -1, -1, -1, -1, -1, -1, -1), (0, 7, 6, -1, -1, -1, -1, -1, -1), (0, 7, 6, 5, -1, -1, -1, -1, -1), (0, 7, 6, 5, 4, -1, -1, -1, -1), (0, 7, 6, 5, 4, 3, -1, -1, -1), (0, 7, 6, 5, 4, 3, 2, -1, -1), (0, 7, 6, 5, 4, 3, 2, 1, -1), (0, 7, 6, 5, 4, 3, 2, 1, 0)]


### Implementación H3(n) - Distancias medias

La tercera de las heurísticas, la de las distancias medias, se ha implementado de la siguiente manera:

Dado un estado, su función heurística h(n) será la distancia media a todas las ciudades que no se han visitado todavía.
Para ello, en primer lugar se comprobará si ya se ha terminado el recorrido o no, es decir, si ya se han visitado todas las ciudades y regresado a la ciudad inicial. En caso afirmativo, se devolverá un valor 0. En caso contrario, pueden darse dos casos: 
- Que se hayan visitado todas las ciudades pero que falte regresar a la ciudad inicial: en ese caso hay una única opción de movimiento y h(n) será la distancia desde la última ciudad visitada a la ciudad de partida.
- Que aún falten ciudades por visitar: en ese caso, se tomará la última ciudad visitada y h(n) será la media de las distancias desde esa ciudad a todas las que aún faltan por visitar. Es decir, si actualmente nos encontramos en la ciudad 1: (0,1,-1,-1,-1,-1,-1,-1,-1), su función heurística será la distancia media de las distancias: [(1,2), (1,3), (1,4), (1,5), (1,6), (1,7), (1,8)].

In [23]:
class ProblemaViajanteComercioH3(Problem):
    """ Problema Viajante de Comercio"""
    
    def __init__(self, localizaciones):
        self.localizaciones = localizaciones
        self.initial = [0] #se empieza el recorrido en la ciudad 0
                 
        lista=list(self.initial) #la tupla del estado inicial se completa con -1 para el resto de ciudades (0,-1,-1,...)                
        for i in range (self.localizaciones.nciudades):
            lista.append(-1) 
        self.initial=tuple(lista)
        
            
    def actions(self, state):
        """devuelve la lista de ciudades no visitadas como acciones potenciales."""     
        #cuando ya se han visitado todas las ciudades, hay que volver a la inicial
        if (state.index(-1)==self.localizaciones.nciudades): 
            return tuple([0]) #la única acción posible es volver a la ciudad inicial
        
        lista=[]
        ciudad=state[state.index(-1)-1] #se obtiene la última ciudad visitada. Ej: (0,3,5,-1,-1,-1)->5     
        for i in range(self.localizaciones.nciudades):
            if i!=ciudad and i not in state: #ciudades que aún no se han visitado
                lista.append(i)
                
        actions=tuple(lista)   
        return actions #se devuelven todas las posibles ciudades a las que se puede ir
         
           
    
    
    def result(self, state, action):
        """produce el nuevo estado al anadir la accion seleccionada"""   
        lista=list(state) 
        lista[state.index(-1)]=action #la accion indica la ciudad a la que se debe ir
        result=tuple(lista)
        
        return result
    
    
    def is_goal(self, state):        
        cont=0
        for i in range(self.localizaciones.nciudades):  #se comprueba que se hayan visitado todas las ciudades
            if i in state:
                cont+=1
              
        #el problema está resuelto si se visitaron todas las ciudades y se ha vuelto a la ciudad inicial
        if cont==(self.localizaciones.nciudades) and (state[0]==state[self.localizaciones.nciudades]):  
            return True
        return False
    
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        if -1 in s1: 
            #se devuelve la distancia entre las dos últimas ciudades
            #s tendrá la forma: (n1, n2, n3, -1, -1...) y la acción indicará que se vaya a n4, por lo tanto:
            #s1 tendrá la forma: (n1, n2, n3, n4, -1...)
            #y lo que se hace es devolver la distancia entre n3 y n4
            return self.localizaciones.distancia(s1[s1.index(-1)-2],s1[s1.index(-1)-1]) 
        else:
            #si ya se han visitado todas las ciudades, se devuelve la distancia entre la última y el regreso a la ciudad inicial
            return self.localizaciones.distancia(s1[self.localizaciones.nciudades-1],s1[self.localizaciones.nciudades]) 
        
        
        
        
    def h(self, node):      
        # ToDo
        
        node=node.state 
        
        if -1 in node: #se comprueba si el recorrido ya ha finalizado   
            if(node.index(-1)==self.localizaciones.nciudades): #se comprueba si se han visitado ya todas las ciudades y solo falta volver a la inicial
                return self.localizaciones.distancia(node[self.localizaciones.nciudades-1], 0)
            
            sum=0.0
            cont=0
            ciudad=node[node.index(-1)-1] #se obtiene la ciduad que se está probando
                        
            for i in range (self.localizaciones.nciudades):
                if i not in node:
                    cont+=1
                    sum+=self.localizaciones.distancia(ciudad, i) #se van acumulando las distancias desde la ciudad que se está probando a todas las que aún quedan por visitar
        
            return sum/cont
                
        else:
            return 0
            
    
               

g1=Localizaciones(filename='./data/grafo8cidades.txt')
VC1=ProblemaViajanteComercioH3(g1)
sol = astar_search(VC1)
print (f'\n\nCoste del camino: {sol.path_cost:f}')
print (f'Estado solución: {sol.state:}')
print (f'Lista de acciones: {path_actions(sol)}')
print (f'Numero de acciones en el camino: {len(path_actions(sol))}')
print (f'Lista de estados:')
print (path_states(sol))
    




Coste del camino: 381.669962
Estado solución: (0, 7, 6, 5, 4, 3, 2, 1, 0)
Lista de acciones: [7, 6, 5, 4, 3, 2, 1, 0]
Numero de acciones en el camino: 8
Lista de estados:
[(0, -1, -1, -1, -1, -1, -1, -1, -1), (0, 7, -1, -1, -1, -1, -1, -1, -1), (0, 7, 6, -1, -1, -1, -1, -1, -1), (0, 7, 6, 5, -1, -1, -1, -1, -1), (0, 7, 6, 5, 4, -1, -1, -1, -1), (0, 7, 6, 5, 4, 3, -1, -1, -1), (0, 7, 6, 5, 4, 3, 2, -1, -1), (0, 7, 6, 5, 4, 3, 2, 1, -1), (0, 7, 6, 5, 4, 3, 2, 1, 0)]


**==========================================COMPARATIVA HEURÍSTICAS==============================================**

In [24]:
'''La solución real a este apartado se encuentra al final del mismo. 
En primer lugar se incluyen las funciones y acciones necesarias para poder realizar la comparación.'''

#implementación del Problema con las tres heurísticas 
class ProblemaViajanteComercioComparativa(Problem):
    """ Problema Viajante de Comercio"""
    
    def __init__(self, localizaciones, goal=()):
        self.localizaciones = localizaciones
        self.initial = [0] #se empieza el recorrido en la ciudad 0
        self.goal=goal
        
        lista=list(self.initial) #la tupla del estado inicial se completa con -1 para el resto de ciudades (0,-1,-1,...)                
        for i in range (self.localizaciones.nciudades):
            lista.append(-1) 
        self.initial=tuple(lista)
        
            
    def actions(self, state):
        """devuelve la lista de ciudades no visitadas como acciones potenciales."""     
        #cuando ya se han visitado todas las ciudades, hay que volver a la inicial
        if (state.index(-1)==self.localizaciones.nciudades): 
            return tuple([0]) #la única acción posible es volver a la ciudad inicial
        
        lista=[]
        ciudad=state[state.index(-1)-1] #se obtiene la última ciudad visitada. Ej: (0,3,5,-1,-1,-1)->5     
        for i in range(self.localizaciones.nciudades):
            if i!=ciudad and i not in state: #ciudades que aún no se han visitado
                lista.append(i)
                
        actions=tuple(lista)   
        return actions #se devuelven todas las posibles ciudades a las que se puede ir
         
           
    
    
    def result(self, state, action):
        """produce el nuevo estado al anadir la accion seleccionada"""   
        lista=list(state) 
        lista[state.index(-1)]=action #la accion indica la ciudad a la que se debe ir
        result=tuple(lista)
        
        return result
    
    
    def is_goal(self, state):        
        cont=0
        for i in range(self.localizaciones.nciudades):  #se comprueba que se hayan visitado todas las ciudades
            if i in state:
                cont+=1
              
        #el problema está resuelto si se visitaron todas las ciudades y se ha vuelto a la ciudad inicial
        if cont==(self.localizaciones.nciudades) and (state[0]==state[self.localizaciones.nciudades]):  
            return True
        return False
    
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        if -1 in s1: 
            #se devuelve la distancia entre las dos últimas ciudades
            #s tendrá la forma: (n1, n2, n3, -1, -1...) y la acción indicará que se vaya a n4, por lo tanto:
            #s1 tendrá la forma: (n1, n2, n3, n4, -1...)
            #y lo que se hace es devolver la distancia entre n3 y n4
            return self.localizaciones.distancia(s1[s1.index(-1)-2],s1[s1.index(-1)-1]) 
        else:
            #si ya se han visitado todas las ciudades, se devuelve la distancia entre la última y el regreso a la ciudad inicial
            return self.localizaciones.distancia(s1[self.localizaciones.nciudades-1],s1[self.localizaciones.nciudades]) 
     
    
    #vecino más próximo
    def h1(self, node): 
        node=node.state

        if -1 in node: #se comprueba si el recorrido ya ha finalizado
            if node.index(-1)==self.localizaciones.nciudades: #se comprueba si se han visitado ya todas las ciudades y solo falta volver a la inicial
                return self.localizaciones.distancia(node[node.index(-1)-1], 0)

            else: 
                ciudad=node[node.index(-1)-1] #se obtiene la ciudad que se está probando
                distancias={}
                for i in range(1, self.localizaciones.nciudades):
                    if i not in node:
                        distancias[i]=self.localizaciones.distancia(ciudad, i) #se obtienen las distancias entre la ciudad que se está probando y las que todavía quedan por visitar

                minimo= min(distancias.values())  #se obtiene el mínimo de todas esas distancias
                return minimo

        else:
            return 0   
        
    
    #vecino más lejano
    def h2(self, node): 
        node=node.state

        if -1 in node: #se comprueba si el recorrido ya ha finalizado
            if node.index(-1)==self.localizaciones.nciudades:  #se comprueba si se han visitado ya todas las ciudades y solo falta volver a la inicial
                return self.localizaciones.distancia(node[node.index(-1)-1], 0) 

            else: 
                ciudad=node[node.index(-1)-1] #se obtiene la ciduad que se está probando
                distancias={}
                for i in range(1, self.localizaciones.nciudades):
                    if i not in node:
                        distancias[i]=self.localizaciones.distancia(ciudad, i) #se obtienen las distancias entre la ciudad que se está probando y las que todavía quedan por visitar

                maximo= max(distancias.values())  #se obtiene el máximo de todas esas distancias
                return maximo
                

        else:
            return 0  
        
        
    #distancias medias  
    def h3(self, node):      
        # ToDo
        
        node=node.state 
        
        if -1 in node: #se comprueba si el recorrido ya ha finalizado   
            if(node.index(-1)==self.localizaciones.nciudades): #se comprueba si se han visitado ya todas las ciudades y solo falta volver a la inicial
                return self.localizaciones.distancia(node[self.localizaciones.nciudades-1], 0)
            
            sum=0.0
            cont=0
            ciudad=node[node.index(-1)-1] #se obtiene la ciduad que se está probando
                        
            for i in range (self.localizaciones.nciudades):
                if i not in node:
                    cont+=1
                    sum+=self.localizaciones.distancia(ciudad, i) #se van acumulando las distancias desde la ciudad que se está probando a todas las que aún quedan por visitar
        
            return sum/cont
                
        else:
            return 0



In [25]:
#COMPARACIÓN H1, H2, H3

#carga de los distintos ficheros
g1=Localizaciones(filename='./data/grafo8cidades.txt')
g2=Localizaciones(filename='./data/grafos10_10/grafo_1.txt')
g3=Localizaciones(filename='./data/grafos10_20/grafo_10.txt')
g4=Localizaciones(filename='./data/grafos11_10/grafo_7.txt')

#definición de las funciones con cada una de las heurísticas
def astarVCH1(problem): return astar_search(problem, h=problem.h1)
def astarVCH2(problem): return astar_search(problem, h=problem.h2)
def astarVCH3(problem): return astar_search(problem, h=problem.h3)

#llamada al problema con cada uno de los ficheros
VC1=ProblemaViajanteComercioComparativa(g1)
VC2=ProblemaViajanteComercioComparativa(g2)
VC3=ProblemaViajanteComercioComparativa(g3)
VC4=ProblemaViajanteComercioComparativa(g4)


#se han modificado ligeramente estas dos funciones para imprimir por pantalla los campos que nos interesan
def reportComp(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print('\n\n', searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_countsComp(counts, str(p)[:40])
        #report_countsComp(total_counts, 'TOTAL\n')

def report_countsComp(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | '.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions']))

reportComp([astarVCH1, astarVCH2, astarVCH3], [VC1, VC2, VC3, VC4])
#(cada fila dentro de cada algoritmo se corresponde con las soluciones del problema del viajante para cada uno de los ficheros de entrada)




 astarVCH1:
    2,369 nodes |      732 goal |  382 cost |     739 actions | 
   71,098 nodes |   20,001 goal |  501 cost |  20,010 actions | 
   44,162 nodes |   11,694 goal |  547 cost |  11,703 actions | 
2,147,966 nodes |1,034,742 goal | 6821 cost |1,034,752 actions | 


 astarVCH2:
      971 nodes |      266 goal |  382 cost |     273 actions | 
   26,612 nodes |    6,842 goal |  501 cost |   6,851 actions | 
   11,284 nodes |    2,687 goal |  547 cost |   2,696 actions | 
2,008,199 nodes |  995,709 goal | 6821 cost | 995,719 actions | 


 astarVCH3:
    1,549 nodes |      449 goal |  382 cost |     456 actions | 
   44,689 nodes |   11,930 goal |  501 cost |  11,939 actions | 
   22,213 nodes |    5,498 goal |  547 cost |   5,507 actions | 
2,055,478 nodes |1,007,969 goal | 6821 cost |1,007,979 actions | 


Como se puede apreciar, las tres heurísticas proporcionan la misma distancia final (columna "cost") para cada uno de los ficheros de entrada. Esto es debido a que el algoritmo astar_search proporciona la solución óptima.

La heurística que proporciona mejores resultados en cuanto a número de nodos visitados y de acciones realizadas es la h2, la del vecino más lejano, seguida por la heurística h3, la de las distancias medias. Por último, la heurística h1 del vecino más próximo genera una mayor cantidad de nodos y realiza un mayor número de acciones. 

Entiendo que esto es así ya que en la heurística h1 es demasiado optimista, es decir, el hecho de que en cada iteración se visite la ciudad más próxima (un mínimo local), no asegura obtener también el mínimo global. Es decir, se comprueba que aunque en cada paso se tome el que parece el camino más prometedor, esto no significa que el recorrido total sea el más eficiente, si no que al final se generan más nodos de los necesarios usando otras heurísticas. 

La heurística h2, sin embargo, es todo lo contrario a la h1. Es decir, en cada paso se toma la acción más pesimista, poniéndose en el peor caso, pero finalmente para realizar el recorrido total se genera una menor cantidad de nodos y acciones.

La heurística h3 se encuentra en una situación intermedia, más realista que las otras dos, ya que en cada paso se visita la ciudad desde la cual la distancia media al resto de ciudades que quedan por visitar sea la menor. 

### Implementación heurísitca basada en el árbol de expansión mínima de Prim

Tal y como sugiere el libro de Russell & Norvig, se puede utilizar una función heurística basada en el árbol de expansión mínima (minimum spanning tree - MST) como subestimación del coste, que se podría obtener mediante los algoritmos de Prim o Kruskal. 

La heurística implementada en este apartado se corresponde con el árbol de expansión mínima de Prim. 

Para ello, dado un nodo, es decir, dado un conjunto de ciudades ya visitadas, se devuelve la distancia obtenida al generar el árbol de expansión mínima desde la ciudad que se está probando.

Por lo tanto, lo que se hace es, partiendo de esa ciudad, se va calculando la ciudad más próxima de todas las que quedan por visitar hasta que se hayan recorrido todas. En ese momento, se añade la distancia desde la última ciudad visitada a la ciudad de partida. Después, se decidirá a qué ciudad ir a continuación en función de esa distancia calculada con el árbol de expansión mínima.

In [27]:
'''La solución real a este apartado se encuentra al final del mismo. 
En primer lugar se incluyen las funciones y acciones necesarias para poder probar este problema.'''

#-------------------------H4------------------------------
class ProblemaViajanteComercioH4(Problem):
    """ Problema Viajante de Comercio"""
    
    def __init__(self, localizaciones, goal=()):
        self.localizaciones = localizaciones
        self.initial = [0] #se empieza el recorrido en la ciudad 0
        self.goal=goal
                 
        lista=list(self.initial) #la tupla del estado inicial se completa con -1 para el resto de ciudades (0,-1,-1,...)                
        for i in range (self.localizaciones.nciudades):
            lista.append(-1) 
        self.initial=tuple(lista)
        
            
    def actions(self, state):
        """devuelve la lista de ciudades no visitadas como acciones potenciales."""     
        #cuando ya se han visitado todas las ciudades, hay que volver a la inicial
        if (state.index(-1)==self.localizaciones.nciudades): 
            return tuple([0]) #la única acción posible es volver a la ciudad inicial
        
        lista=[]
        ciudad=state[state.index(-1)-1] #se obtiene la última ciudad visitada. Ej: (0,3,5,-1,-1,-1)->5     
        for i in range(self.localizaciones.nciudades):
            if i!=ciudad and i not in state: #ciudades que aún no se han visitado
                lista.append(i)
                
        actions=tuple(lista)   
        return actions #se devuelven todas las posibles ciudades a las que se puede ir
         
             
    
    def result(self, state, action):
        """produce el nuevo estado al anadir la accion seleccionada"""   
        lista=list(state) 
        lista[state.index(-1)]=action #la accion indica la ciudad a la que se debe ir
        result=tuple(lista)
        
        return result
    
    
    def is_goal(self, state):        
        cont=0
        for i in range(self.localizaciones.nciudades):  #se comprueba que se hayan visitado todas las ciudades
            if i in state:
                cont+=1
              
        #el problema está resuelto si se visitaron todas las ciudades y se ha vuelto a la ciudad inicial
        if cont==(self.localizaciones.nciudades) and (state[0]==state[self.localizaciones.nciudades]):  
            return True
        return False
    
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""   
        if -1 in s1: 
            #se devuelve la distancia entre las dos últimas ciudades
            #s tendrá la forma: (n1, n2, n3, -1, -1...) y la acción indicará que se vaya a n4, por lo tanto:
            #s1 tendrá la forma: (n1, n2, n3, n4, -1...)
            #y lo que se hace es devolver la distancia entre n3 y n4
            return self.localizaciones.distancia(s1[s1.index(-1)-2],s1[s1.index(-1)-1]) 
        else:
            #si ya se han visitado todas las ciudades, se devuelve la distancia entre la última y el regreso a la ciudad inicial
            return self.localizaciones.distancia(s1[self.localizaciones.nciudades-1],s1[self.localizaciones.nciudades]) 
        
        
        
    #árbol de expansión mínima: algoritmo de Prim
    def h(self, node):      
        # ToDo
        
        node=node.state 
            
        if -1 in node: #se comprueba si el recorrido ya ha finalizado
            if(node.index(-1)==self.localizaciones.nciudades): #se comprueba si se han visitado ya todas las ciudades y solo falta volver a la inicial
                return self.localizaciones.distancia(node[self.localizaciones.nciudades-1], 0)
            
            else:
                ciudad=node[node.index(-1)-1] #se obtiene la ciudad que se está probando
                sum=0.0
                #número de ciudades que ya se visitaron
                visit=node.index(-1)
                                
                #cálculo del árbol de expansión mínima con el algoritmo de Prim    
                visitadas=list()

                #se obtiene la ciudad más cercana a la que se está probando
                ciudad=node[node.index(-1)-1]
                distancias={}
                for i in range(1, self.localizaciones.nciudades):
                    if i not in node:
                        distancias[i]=self.localizaciones.distancia(ciudad, i)

                minimo= min(distancias.values())                
                visitadas.append(min(distancias, key=distancias.get))
                sum+=minimo
                
                ciudad=visitadas[len(visitadas)-1]
                
                
                #resto de ciudades
                while len(visitadas)+visit<self.localizaciones.nciudades: #mientras no se hayan visitado todas las ciudades:
                    distancias={}
                    for i in range(1, self.localizaciones.nciudades):
                        if i not in node and i not in visitadas:
                            distancias[i]=self.localizaciones.distancia(ciudad, i)
                    
                    minimo= min(distancias.values())
                    sum+=minimo
                    visitadas.append(min(distancias, key=distancias.get))
                    ciudad=visitadas[len(visitadas)-1]
                    
                    
                #vuelta a la ciudad inicial
                sum+=self.localizaciones.distancia(visitadas[len(visitadas)-1], 0)                
                
                return sum
            
        else:
            return 0     
            

            

#llamada al problema con cada uno de los ficheros
VC1=ProblemaViajanteComercioH4(g1)
VC2=ProblemaViajanteComercioH4(g2)
VC3=ProblemaViajanteComercioH4(g3)
VC4=ProblemaViajanteComercioH4(g4)


reportComp([astar_search], [VC1, VC2, VC3, VC4])

#(cada fila dentro de cada algoritmo se corresponde con las soluciones del problema del viajante usando 
#la heurística del árbol de expansión mínima de Prim para cada uno de los ficheros de entrada)



 astar_search:
       29 nodes |        9 goal |  382 cost |      16 actions | 
       46 nodes |       11 goal |  576 cost |      20 actions | 
       46 nodes |       11 goal |  547 cost |      20 actions | 
       56 nodes |       12 goal | 6821 cost |      22 actions | 


Aquí se puede ver la ejecución del problema del viajante usando la heurística del árbol de expansión mínima de Prim. 

Se puede apreciar como, para cualquiera de los ficheros de entrada, tanto el número de nodos generados como de acciones realizadas es notablemente menor que con cualquiera de las tres heurísticas implementadas en el apartado anterior. Esto se debe a que se trata de la heurística más informada de todas, ya que para cada ciudad, se calcula cuánta distancia queda para completar todo el recorrido, y se va a esa cuya distancia sea menor.